no need to delete the Nan value when interpolating

In [1]:
import numpy as np
import os
import math
import pandas as pd
import scipy.stats as stat
import scipy.interpolate
from scipy.stats import norm
import matplotlib.pyplot as plt

## 1. Data Preprocessing:

In [2]:
sofr = pd.read_excel("hist_data.xlsm", sheet_name = "SofrCurve")
AAPL = pd.read_excel("hist_data.xlsm", sheet_name = "AAPL").set_index('Date')
MSFT = pd.read_excel("hist_data.xlsm", sheet_name = "MSFT").set_index('Date')
F = pd.read_excel("hist_data.xlsm", sheet_name = "F").set_index('Date')
BAC = pd.read_excel("hist_data.xlsm", sheet_name = "BAC").set_index('Date')

In [3]:
sofr

,Tenor,T,2022-10-31 00:00:00,2022-11-01 00:00:00,2022-11-02 00:00:00,2022-11-03 00:00:00,2022-11-04 00:00:00,2022-11-07 00:00:00,2022-11-08 00:00:00,2022-11-09 00:00:00,...,2023-10-17 00:00:00,2023-10-18 00:00:00,2023-10-19 00:00:00,2023-10-20 00:00:00,2023-10-23 00:00:00,2023-10-24 00:00:00,2023-10-25 00:00:00,2023-10-26 00:00:00,2023-10-27 00:00:00,2023-10-30 00:00:00
0,1D,0.002778,0.039191,0.039604,0.039948,0.040389,0.045965,0.040224,0.040339,0.040373,...,0.052964,0.053038,0.053050,0.053072,0.053090,0.053105,0.052981,0.053047,0.052989,0.052967
1,1M,0.083333,0.038721,0.039023,0.039286,0.039585,0.042343,0.039258,0.039211,0.039193,...,0.053114,0.053102,0.053086,0.053046,0.053077,0.053084,0.053057,0.053089,0.053040,0.053053
2,2M,0.166667,0.038670,0.038886,0.039100,0.039350,0.038795,0.039460,0.039420,0.039483,...,0.053374,0.053284,0.053226,0.053128,0.053191,0.053202,0.053259,0.053218,0.053185,0.053265
3,3M,0.250000,0.040536,0.040725,0.040852,0.041154,0.040611,0.041424,0.041406,0.041445,...,0.053666,0.053561,0.053454,0.053323,0.053404,0.053424,0.053490,0.053414,0.053368,0.053476
4,6M,0.500000,0.044577,0.044849,0.044884,0.045281,0.045212,0.045455,0.045334,0.045299,...,0.054078,0.053989,0.053758,0.053546,0.053638,0.053690,0.053749,0.053567,0.053486,0.053612
5,9M,0.750000,0.046004,0.046448,0.046580,0.047107,0.046752,0.047189,0.046941,0.046883,...,0.053831,0.053753,0.053427,0.053126,0.053211,0.053293,0.053393,0.053089,0.052991,0.053120
6,1Y,1.000000,0.046449,0.046970,0.047203,0.047894,0.047500,0.047929,0.047589,0.047479,...,0.053225,0.053163,0.052750,0.052348,0.052389,0.052503,0.052653,0.052243,0.052115,0.052245
7,2Y,2.000000,0.044583,0.045022,0.045496,0.046594,0.046097,0.046633,0.046046,0.045585,...,0.049497,0.049611,0.049204,0.048366,0.048027,0.048399,0.048791,0.048044,0.047758,0.047904
8,3Y,3.000000,0.042002,0.042344,0.042749,0.043833,0.043385,0.043884,0.043209,0.042645,...,0.046978,0.047304,0.047169,0.046151,0.045582,0.045999,0.046595,0.045645,0.045284,0.045429
9,4Y,4.000000,0.040318,0.040614,0.040868,0.041825,0.041503,0.041916,0.041182,0.040704,...,0.045605,0.046034,0.046195,0.045156,0.044508,0.044850,0.045594,0.044538,0.044200,0.044345


In [4]:
#AAPL.shape

In [5]:
AAPL=AAPL.rename(columns={'Adj Close':'AAPL Adj Close'})
MSFT=MSFT.rename(columns={'Adj Close':'MSFT Adj Close'})
F=F.rename(columns={'Adj Close':'F Adj Close'})
BAC=BAC.rename(columns={'Adj Close':'BAC Adj Close'})

In [6]:
allstockdata = pd.concat([AAPL, MSFT, F, BAC], axis=1)
allstockdata

,AAPL Adj Close,MSFT Adj Close,F Adj Close,BAC Adj Close
Date,,,,
2022-10-31,152.041122,229.443207,11.974380,34.748581
2022-11-01,149.373917,225.529037,12.001248,34.902847
2022-11-02,143.801514,217.552444,11.696740,34.796783
2022-11-03,137.703613,211.770157,11.875863,34.603954
2022-11-04,137.435455,218.827515,12.099767,35.471703
...,...,...,...,...
2023-10-24,172.991058,329.860504,11.215128,25.268324
2023-10-25,170.657135,339.979980,11.362824,25.347691
2023-10-26,166.458023,327.225861,11.175742,25.913177


## 2. Four Stocks & SOFR Swap Risk Factor:

### 2-1. 4 Stocks Risk Factor $ \Delta x_{0,1d}^{(k)} $: daily relative return of the stock prices:

In [7]:
allstock_dailyreturn=np.log(allstockdata / allstockdata.shift(1)).rename(columns=
                                                                         {'AAPL Adj Close':'AAPL daily return',
                                                                          'MSFT Adj Close':'MSFT daily return',
                                                                          'F Adj Close':'F daily return',
                                                                          'BAC Adj Close':'BAC daily return'}).dropna()
allstock_dailyreturn

,AAPL daily return,MSFT daily return,F daily return,BAC daily return
Date,,,,
2022-11-01,-0.017698,-0.017207,0.002241,0.004430
2022-11-02,-0.038019,-0.036009,-0.025700,-0.003043
2022-11-03,-0.043330,-0.026938,0.015198,-0.005557
2022-11-04,-0.001949,0.032782,0.018678,0.024767
2022-11-07,0.003895,0.028850,0.013966,0.005962
...,...,...,...,...
2023-10-24,0.002540,0.003668,-0.007871,-0.003919
2023-10-25,-0.013583,0.030217,0.013083,0.003136
2023-10-26,-0.024913,-0.038236,-0.016601,0.022064


In [8]:
#nan=allstock_dailyreturn.isna().sum()
#nan

### 2-2. SOFR risk factor $ \Delta x_{0,1d}^{(k)} $: absolute daily changes in the zero rate risk factors (all the 30 zero rates) 
##### Hint from prof: This is because the value of the swap depends only on discount factors of the SOFR curve which are all determined by the 30 zero rates.

In [9]:
#sofr.shape

In [10]:
sofr.head() # sofr zero rates

,Tenor,T,2022-10-31 00:00:00,2022-11-01 00:00:00,2022-11-02 00:00:00,2022-11-03 00:00:00,2022-11-04 00:00:00,2022-11-07 00:00:00,2022-11-08 00:00:00,2022-11-09 00:00:00,...,2023-10-17 00:00:00,2023-10-18 00:00:00,2023-10-19 00:00:00,2023-10-20 00:00:00,2023-10-23 00:00:00,2023-10-24 00:00:00,2023-10-25 00:00:00,2023-10-26 00:00:00,2023-10-27 00:00:00,2023-10-30 00:00:00
0,1D,0.002778,0.039191,0.039604,0.039948,0.040389,0.045965,0.040224,0.040339,0.040373,...,0.052964,0.053038,0.053050,0.053072,0.053090,0.053105,0.052981,0.053047,0.052989,0.052967
1,1M,0.083333,0.038721,0.039023,0.039286,0.039585,0.042343,0.039258,0.039211,0.039193,...,0.053114,0.053102,0.053086,0.053046,0.053077,0.053084,0.053057,0.053089,0.053040,0.053053
2,2M,0.166667,0.038670,0.038886,0.039100,0.039350,0.038795,0.039460,0.039420,0.039483,...,0.053374,0.053284,0.053226,0.053128,0.053191,0.053202,0.053259,0.053218,0.053185,0.053265
3,3M,0.250000,0.040536,0.040725,0.040852,0.041154,0.040611,0.041424,0.041406,0.041445,...,0.053666,0.053561,0.053454,0.053323,0.053404,0.053424,0.053490,0.053414,0.053368,0.053476
4,6M,0.500000,0.044577,0.044849,0.044884,0.045281,0.045212,0.045455,0.045334,0.045299,...,0.054078,0.053989,0.053758,0.053546,0.053638,0.053690,0.053749,0.053567,0.053486,0.053612


In [11]:
sofr=sofr.set_index("Tenor").transpose().drop("T") #set tenor as index for later merging
sofr.index = pd.to_datetime(sofr.index)
sofr.head()

Tenor,1D,1M,2M,3M,6M,9M,1Y,2Y,3Y,4Y,...,15Y,16Y,17Y,18Y,19Y,20Y,25Y,30Y,35Y,40Y
2022-10-31,0.039191,0.038721,0.038670,0.040536,0.044577,0.046004,0.046449,0.044583,0.042002,0.040318,...,0.037151,0.037057,0.036907,0.036698,0.036433,0.036111,0.034091,0.032350,0.030552,0.028708
2022-11-01,0.039604,0.039023,0.038886,0.040725,0.044849,0.046448,0.046970,0.045022,0.042344,0.040614,...,0.036802,0.036682,0.036511,0.036287,0.036010,0.035678,0.033645,0.031979,0.030238,0.028478
2022-11-02,0.039948,0.039286,0.039100,0.040852,0.044884,0.046580,0.047203,0.045496,0.042749,0.040868,...,0.036855,0.036701,0.036498,0.036248,0.035953,0.035613,0.033627,0.031936,0.030292,0.028608
2022-11-03,0.040389,0.039585,0.039350,0.041154,0.045281,0.047107,0.047894,0.046594,0.043833,0.041825,...,0.037221,0.037069,0.036886,0.036657,0.036372,0.036022,0.033811,0.032134,0.030407,0.028655
2022-11-04,0.045965,0.042343,0.038795,0.040611,0.045212,0.046752,0.047500,0.046097,0.043385,0.041503,...,0.037687,0.037557,0.037380,0.037152,0.036870,0.036534,0.034424,0.032558,0.030723,0.028933


In [12]:
sofr.shape

(251, 30)

### sofr absolute daily change:

In [13]:
sofr

Tenor,1D,1M,2M,3M,6M,9M,1Y,2Y,3Y,4Y,...,15Y,16Y,17Y,18Y,19Y,20Y,25Y,30Y,35Y,40Y
2022-10-31,0.039191,0.038721,0.038670,0.040536,0.044577,0.046004,0.046449,0.044583,0.042002,0.040318,...,0.037151,0.037057,0.036907,0.036698,0.036433,0.036111,0.034091,0.032350,0.030552,0.028708
2022-11-01,0.039604,0.039023,0.038886,0.040725,0.044849,0.046448,0.046970,0.045022,0.042344,0.040614,...,0.036802,0.036682,0.036511,0.036287,0.036010,0.035678,0.033645,0.031979,0.030238,0.028478
2022-11-02,0.039948,0.039286,0.039100,0.040852,0.044884,0.046580,0.047203,0.045496,0.042749,0.040868,...,0.036855,0.036701,0.036498,0.036248,0.035953,0.035613,0.033627,0.031936,0.030292,0.028608
2022-11-03,0.040389,0.039585,0.039350,0.041154,0.045281,0.047107,0.047894,0.046594,0.043833,0.041825,...,0.037221,0.037069,0.036886,0.036657,0.036372,0.036022,0.033811,0.032134,0.030407,0.028655
2022-11-04,0.045965,0.042343,0.038795,0.040611,0.045212,0.046752,0.047500,0.046097,0.043385,0.041503,...,0.037687,0.037557,0.037380,0.037152,0.036870,0.036534,0.034424,0.032558,0.030723,0.028933
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-24,0.053105,0.053084,0.053202,0.053424,0.053690,0.053293,0.052503,0.048399,0.045999,0.044850,...,0.044093,0.044056,0.043972,0.043841,0.043663,0.043437,0.041874,0.040238,0.038438,0.036493
2023-10-25,0.052981,0.053057,0.053259,0.053490,0.053749,0.053393,0.052653,0.048791,0.046595,0.045594,...,0.045289,0.045268,0.045198,0.045078,0.044906,0.044683,0.043076,0.041358,0.039564,0.037641
2023-10-26,0.053047,0.053089,0.053218,0.053414,0.053567,0.053089,0.052243,0.048044,0.045645,0.044538,...,0.044269,0.044244,0.044171,0.044049,0.043882,0.043668,0.042182,0.040589,0.038760,0.036764
2023-10-27,0.052989,0.053040,0.053185,0.053368,0.053486,0.052991,0.052115,0.047758,0.045284,0.044200,...,0.044460,0.044461,0.044411,0.044309,0.044157,0.043955,0.042508,0.040985,0.039167,0.037148


In [14]:
# sofr_dailychange = abs(sofr - sofr.shift(1)).dropna()  # 使用絕對值
sofr_dailychange = sofr.diff().dropna()
sofr_dailychange

Tenor,1D,1M,2M,3M,6M,9M,1Y,2Y,3Y,4Y,...,15Y,16Y,17Y,18Y,19Y,20Y,25Y,30Y,35Y,40Y
2022-11-01,0.000413,0.000302,0.000216,0.000188,0.000272,0.000444,0.000521,0.000439,0.000342,0.000296,...,-0.000349,-0.000375,-0.000395,-0.000411,-0.000423,-0.000434,-0.000446,-0.000372,-0.000314,-0.000230
2022-11-02,0.000344,0.000262,0.000214,0.000128,0.000035,0.000132,0.000234,0.000474,0.000404,0.000254,...,0.000053,0.000019,-0.000013,-0.000040,-0.000058,-0.000064,-0.000018,-0.000043,0.000054,0.000130
2022-11-03,0.000440,0.000299,0.000250,0.000302,0.000397,0.000527,0.000690,0.001098,0.001085,0.000957,...,0.000366,0.000368,0.000388,0.000409,0.000420,0.000409,0.000184,0.000198,0.000115,0.000047
2022-11-04,0.005576,0.002758,-0.000555,-0.000543,-0.000069,-0.000355,-0.000394,-0.000497,-0.000448,-0.000322,...,0.000466,0.000488,0.000494,0.000495,0.000498,0.000512,0.000613,0.000424,0.000315,0.000278
2022-11-07,-0.005741,-0.003085,0.000665,0.000813,0.000243,0.000437,0.000429,0.000536,0.000499,0.000413,...,0.000431,0.000463,0.000487,0.000504,0.000512,0.000514,0.000498,0.000577,0.000577,0.000562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-24,0.000015,0.000007,0.000011,0.000019,0.000052,0.000083,0.000114,0.000371,0.000417,0.000343,...,-0.000161,-0.000174,-0.000183,-0.000189,-0.000196,-0.000203,-0.000255,-0.000298,-0.000283,-0.000242
2023-10-25,-0.000124,-0.000027,0.000057,0.000066,0.000059,0.000100,0.000150,0.000393,0.000596,0.000743,...,0.001196,0.001212,0.001226,0.001237,0.001244,0.001246,0.001202,0.001119,0.001126,0.001147
2023-10-26,0.000066,0.000032,-0.000041,-0.000076,-0.000182,-0.000304,-0.000410,-0.000747,-0.000950,-0.001055,...,-0.001020,-0.001024,-0.001027,-0.001028,-0.001025,-0.001015,-0.000895,-0.000769,-0.000804,-0.000877
2023-10-27,-0.000059,-0.000049,-0.000033,-0.000046,-0.000082,-0.000099,-0.000129,-0.000286,-0.000361,-0.000339,...,0.000191,0.000217,0.000240,0.000259,0.000275,0.000287,0.000327,0.000396,0.000407,0.000384


In [15]:
#check_na=sofr_dailychange.isna().sum()
#check_na

## 2-3. Total - 34 risk factors $ \color{blue}\Delta x_{0,1d}^{(k)} $ :

$$
\text{L} = P_{1d} - P_0 = \sum_{k=1}^{34} a^{(k)} \cdot  \color{blue}\Delta X_{0,1d}^{(k)}
$$

In [38]:
allriskfactors=pd.merge(allstock_dailyreturn,sofr_dailychange, left_index=True, right_index=True, how='outer')
allriskfactors # merge stocks daily return & swap daily change

,AAPL daily return,MSFT daily return,F daily return,BAC daily return,1D,1M,2M,3M,6M,9M,...,15Y,16Y,17Y,18Y,19Y,20Y,25Y,30Y,35Y,40Y
2022-11-01,-0.017698,-0.017207,0.002241,0.004430,0.000413,0.000302,0.000216,0.000188,0.000272,0.000444,...,-0.000349,-0.000375,-0.000395,-0.000411,-0.000423,-0.000434,-0.000446,-0.000372,-0.000314,-0.000230
2022-11-02,-0.038019,-0.036009,-0.025700,-0.003043,0.000344,0.000262,0.000214,0.000128,0.000035,0.000132,...,0.000053,0.000019,-0.000013,-0.000040,-0.000058,-0.000064,-0.000018,-0.000043,0.000054,0.000130
2022-11-03,-0.043330,-0.026938,0.015198,-0.005557,0.000440,0.000299,0.000250,0.000302,0.000397,0.000527,...,0.000366,0.000368,0.000388,0.000409,0.000420,0.000409,0.000184,0.000198,0.000115,0.000047
2022-11-04,-0.001949,0.032782,0.018678,0.024767,0.005576,0.002758,-0.000555,-0.000543,-0.000069,-0.000355,...,0.000466,0.000488,0.000494,0.000495,0.000498,0.000512,0.000613,0.000424,0.000315,0.000278
2022-11-07,0.003895,0.028850,0.013966,0.005962,-0.005741,-0.003085,0.000665,0.000813,0.000243,0.000437,...,0.000431,0.000463,0.000487,0.000504,0.000512,0.000514,0.000498,0.000577,0.000577,0.000562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-24,0.002540,0.003668,-0.007871,-0.003919,0.000015,0.000007,0.000011,0.000019,0.000052,0.000083,...,-0.000161,-0.000174,-0.000183,-0.000189,-0.000196,-0.000203,-0.000255,-0.000298,-0.000283,-0.000242
2023-10-25,-0.013583,0.030217,0.013083,0.003136,-0.000124,-0.000027,0.000057,0.000066,0.000059,0.000100,...,0.001196,0.001212,0.001226,0.001237,0.001244,0.001246,0.001202,0.001119,0.001126,0.001147
2023-10-26,-0.024913,-0.038236,-0.016601,0.022064,0.000066,0.000032,-0.000041,-0.000076,-0.000182,-0.000304,...,-0.001020,-0.001024,-0.001027,-0.001028,-0.001025,-0.001015,-0.000895,-0.000769,-0.000804,-0.000877
2023-10-27,0.007938,0.005839,-0.130641,-0.037049,-0.000059,-0.000049,-0.000033,-0.000046,-0.000082,-0.000099,...,0.000191,0.000217,0.000240,0.000259,0.000275,0.000287,0.000327,0.000396,0.000407,0.000384


In [17]:
rows_with_na = allriskfactors[allriskfactors.isna().any(axis=1)]
rows_with_na

,AAPL daily return,MSFT daily return,F daily return,BAC daily return,1D,1M,2M,3M,6M,9M,...,15Y,16Y,17Y,18Y,19Y,20Y,25Y,30Y,35Y,40Y
2022-11-11,0.019085,0.016854,0.022316,0.007316,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-04-07,NaN,NaN,NaN,NaN,0.000596,-0.000189,-0.001362,-0.001413,0.000175,0.000699,...,0.000680,0.000667,0.000657,0.000647,0.000637,0.000626,0.000542,0.000435,0.000439,0.000472
2023-06-19,NaN,NaN,NaN,NaN,0.000035,0.000088,0.000109,0.000121,0.000143,0.000175,...,-0.000029,-0.000033,-0.000035,-0.000037,-0.000039,-0.000039,-0.000035,-0.000034,-0.000029,-0.000024
2023-10-09,0.008416,0.007792,0.005816,0.009164,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### ＊date with no data in stock daily return: 2023-04-07, 2023-06-19  -> interpolation
### ＊date with no data in sofr absolute daily change  : 2022-11-11, 2023-10-09  -> interpolation

## use interpolation to fill NA value and get final 34 risk factors $ \color{blue}\Delta x_{0,1d}^{(34)} $ :

In [18]:
allriskfactors = pd.concat([allstock_dailyreturn, sofr_dailychange], axis=1).loc['2022-10-31':'2023-10-30']
allriskfactors = allriskfactors.interpolate(method='linear', limit_direction='both')

In [19]:
allriskfactors #252 rows

,AAPL daily return,MSFT daily return,F daily return,BAC daily return,1D,1M,2M,3M,6M,9M,...,15Y,16Y,17Y,18Y,19Y,20Y,25Y,30Y,35Y,40Y
2022-11-01,-0.017698,-0.017207,0.002241,0.004430,0.000413,0.000302,0.000216,0.000188,0.000272,0.000444,...,-0.000349,-0.000375,-0.000395,-0.000411,-0.000423,-0.000434,-0.000446,-0.000372,-0.000314,-0.000230
2022-11-02,-0.038019,-0.036009,-0.025700,-0.003043,0.000344,0.000262,0.000214,0.000128,0.000035,0.000132,...,0.000053,0.000019,-0.000013,-0.000040,-0.000058,-0.000064,-0.000018,-0.000043,0.000054,0.000130
2022-11-03,-0.043330,-0.026938,0.015198,-0.005557,0.000440,0.000299,0.000250,0.000302,0.000397,0.000527,...,0.000366,0.000368,0.000388,0.000409,0.000420,0.000409,0.000184,0.000198,0.000115,0.000047
2022-11-04,-0.001949,0.032782,0.018678,0.024767,0.005576,0.002758,-0.000555,-0.000543,-0.000069,-0.000355,...,0.000466,0.000488,0.000494,0.000495,0.000498,0.000512,0.000613,0.000424,0.000315,0.000278
2022-11-07,0.003895,0.028850,0.013966,0.005962,-0.005741,-0.003085,0.000665,0.000813,0.000243,0.000437,...,0.000431,0.000463,0.000487,0.000504,0.000512,0.000514,0.000498,0.000577,0.000577,0.000562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-24,0.002540,0.003668,-0.007871,-0.003919,0.000015,0.000007,0.000011,0.000019,0.000052,0.000083,...,-0.000161,-0.000174,-0.000183,-0.000189,-0.000196,-0.000203,-0.000255,-0.000298,-0.000283,-0.000242
2023-10-25,-0.013583,0.030217,0.013083,0.003136,-0.000124,-0.000027,0.000057,0.000066,0.000059,0.000100,...,0.001196,0.001212,0.001226,0.001237,0.001244,0.001246,0.001202,0.001119,0.001126,0.001147
2023-10-26,-0.024913,-0.038236,-0.016601,0.022064,0.000066,0.000032,-0.000041,-0.000076,-0.000182,-0.000304,...,-0.001020,-0.001024,-0.001027,-0.001028,-0.001025,-0.001015,-0.000895,-0.000769,-0.000804,-0.000877
2023-10-27,0.007938,0.005839,-0.130641,-0.037049,-0.000059,-0.000049,-0.000033,-0.000046,-0.000082,-0.000099,...,0.000191,0.000217,0.000240,0.000259,0.000275,0.000287,0.000327,0.000396,0.000407,0.000384


In [20]:
row_with_na = allriskfactors[allriskfactors.isna().any(axis=1)]
row_with_na

,AAPL daily return,MSFT daily return,F daily return,BAC daily return,1D,1M,2M,3M,6M,9M,...,15Y,16Y,17Y,18Y,19Y,20Y,25Y,30Y,35Y,40Y


In [21]:
allriskfactors.to_csv('allriskfactors.csv') #save csv

## 3-1. Find SOFR Swap Present Value to get the initial portfolio value :
(Swap's Maturity=10Y, so only filter 1Y~10Y zero rates to discount, the result will be the same if calculating it using Prof's 'Swap Pricing Example Excel'):


$$
DF(T) = e^{-r(T) \times T}
$$

$$
PV_{\text{fixed}} = \sum_{T=1}^{10} \left( CF_{\text{fixed},T} \times DF(T) \right)
$$

$$
PV_{\text{float}} = \sum_{T=1}^{10} \left( CF_{\text{float},T} \times DF(T) \right)
$$

$$
PV_{\text{swap}} = PV_{\text{float}} - PV_{\text{fixed}}
$$



In [22]:
matching_tenors = ['1Y', '2Y', '3Y', '4Y', '5Y', '6Y', '7Y', '8Y', '9Y', '10Y']
tenors_sofr = ['1D'] + matching_tenors
latest_date = '2023-10-30'
sofr_rates = sofr.loc[latest_date].astype(float)
swap_notional = 100e6
strike = 0.042
tenors = np.arange(1, 11)

def calculate_swap_pv(rates, tenors, notional, strike):
    discount_factors = np.exp(-rates * tenors)
    float_leg = notional * (1 - discount_factors[-1]) #float leg formula 
    fixed_cashflows = notional * strike 
    fix_leg = np.sum(fixed_cashflows * discount_factors) #fix leg formul
    return float_leg - fix_leg

base_rates = sofr_rates[matching_tenors].values
base_pv = calculate_swap_pv(base_rates, tenors, swap_notional, strike)
print(f"Swap Present Value PV: ${base_pv}")

Swap Present Value PV: $2442901.9998463243


## 3-1. SOFR Swap PV01 (include all 30Yr zero rate, because PV01 reflects 1bp shift impact in the whole SOFR curve):

$$
PV01 = \frac{P_{\text{SOFR Swap, r+1bp}} - P_{\text{SOFR Swap, r}}}{0.0001}
$$

In [40]:
base_pv

np.float64(2442901.9998463243)

In [25]:
def calculate_swap_sensitivity(rates, notional, strike):
    tenors_names = ['1Y', '2Y', '3Y', '4Y', '5Y', '6Y', '7Y', '8Y', '9Y', '10Y'] #~10Y maturity
    tenors_values = np.arange(1, 11) 
    original_price_df = pd.DataFrame(index=rates.index, columns=['Swap Base Price'])
    sensitivity_swap_factor = {}
    
    for start_date in rates.index:
        base_rates = rates.loc[start_date, matching_tenors].astype(float).values
        original_price_df.loc[start_date, 'Swap Base Price'] = calculate_swap_pv(base_rates, tenors_values, notional, strike)
        sensitivity = pd.Series(0.0, index=tenors_names)
        
        for tenor in tenors_names:
            shifted_rates = rates.loc[start_date].astype(float).copy()
            shifted_rates[tenor] += 0.0001 #1bp
            if tenor == '1D':
                sensitivity[tenor] = 0.0
            else:
                bumped_pv = calculate_swap_pv(shifted_rates[matching_tenors].values, tenors_values, notional, strike)
                sensitivity[tenor] = (bumped_pv - original_price_df.loc[start_date, 'Swap Base Price']) / 0.0001 * 10000  
        sensitivity_swap_factor[start_date] = sensitivity
    
    return pd.DataFrame.from_dict(sensitivity_swap_factor, orient='index'), original_price_df

sofr_sensitivity, sofr_price = calculate_swap_sensitivity(sofr, swap_notional, strike)
print("SOFR Sensitivity:", sofr_sensitivity.values)

SOFR Sensitivity: [[4.00917601e+10 7.68266074e+10 1.11066086e+11 ... 2.48521099e+11
  2.69686687e+11 7.16853980e+12]
 [4.00708803e+10 7.67591907e+10 1.10952226e+11 ... 2.48606529e+11
  2.69922820e+11 7.17885311e+12]
 [4.00615145e+10 7.66864591e+10 1.10817669e+11 ... 2.48410367e+11
  2.69704393e+11 7.17229240e+12]
 ...
 [3.98601139e+10 7.62966472e+10 1.09858937e+11 ... 2.36595790e+11
  2.54673108e+11 6.71435345e+12]
 [3.98652488e+10 7.63403581e+10 1.09977883e+11 ... 2.36771172e+11
  2.54757190e+11 6.71364820e+12]
 [3.98600677e+10 7.63179678e+10 1.09930011e+11 ... 2.36532276e+11
  2.54477691e+11 6.70595263e+12]]


## 3-2. Sensitivity $ a^{(k)} $ weight factor:

- SOFR Swap Sensitivity:
   PV01 (Price Value of 1 Basis Point) to measure rate sensitivity.
  $$
a(k) =
\begin{cases}
PV01_{\text{SOFR Swap}}, & k \in \text{SOFR 30 Tenors} \\
1M, & k \in \text{4 Stocks}
\end{cases} $$

- Stock Sensitivities:
   $$ a^{(k)} = \text{Shares Held} \times \text{Current Stock Price} $$
   each stock $1M$, $ a^{(k)} = 1M $.

- Portfolio $ a^{(k)}$ weight factor:
$$ a(k) =
\begin{bmatrix}
PV01_{1D} \\
PV01_{1M} \\
PV01_{2M} \\
\vdots \\
PV01_{30Y} \\
1M \\
1M \\
1M \\
1M
\end{bmatrix}, k=34
$$


## 3-3. Initial Portfolio Total Value $ P_0 $:

 total initial value of the portfolio (for calculating P&L):
   $$ P_0 = P_{\text{SOFR Swap}} + 1M(AAPL) + 1M(MSFT) + 1M(F) + 1M(BAC) $$



In [41]:
base_pv # from 3-1. SOFR Swap Present Value
stock=4000000
P0 = base_pv +stock # swap pv + 4M stocks

print(f"Swap Initial Present Value : ${base_pv}")
print(f"Initial Total Portfolio Value (P0): ${P0}")

Swap Initial Present Value : $2442901.9998463243
Initial Total Portfolio Value (P0): $6442901.999846324


## 3-4. Portfolio Mean $ \mu_P $:

 $$
  \mu_P = \sum_{k=1}^{m} a^{(k)} \cdot \mu_k \cdot 1
  $$
  $ \mu_k $ = Mean of daily changes in each risk factor.
   $ h = 1 $ day 


In [42]:
stock_sensitivity = np.array([1e6] * 4)
relevant_factors = ['AAPL daily return', 'MSFT daily return', 'F daily return', 'BAC daily return'] + matching_tenors  
ak_sensitivity = np.concatenate([sofr_sensitivity[matching_tenors].mean() / 10000, stock_sensitivity])  

print("Sensitivity Vector:", ak_sensitivity)

Sensitivity Vector: [3.99716547e+06 7.69066531e+06 1.11655970e+07 1.44255919e+07
 1.74700654e+07 2.03058689e+07 2.29387143e+07 2.53711812e+07
 2.76083034e+07 7.35762223e+08 1.00000000e+06 1.00000000e+06
 1.00000000e+06 1.00000000e+06]


In [43]:
daily_swap_pnl = (sofr_sensitivity[matching_tenors].shift(1) / 10000 * sofr_dailychange[matching_tenors]).sum(axis=1).dropna()  
mean_swap = daily_swap_pnl.mean() #sofr swap mean

mean_stocks = np.dot(stock_sensitivity, allriskfactors[relevant_factors[:4]].mean())
portfolio_mean = mean_stocks + mean_swap

print("Stock Mean:", mean_stocks)
print("Swap Mean:", mean_swap)
print("Portfolio Mean:", portfolio_mean)

Stock Mean: -139.33001705224967
Swap Mean: 23599.195944200197
Portfolio Mean: 23459.86592714795


## 3-5. Portfolio Variance $ \sigma_P^2 $


$$
  \sigma_P^2 = V\left[ \sum_{k=1}^{m} a^{(k)} \cdot \Delta x_{0,1d}^{(k)} \right] =  w \cdot \Sigma h \cdot w^T 
=  a \cdot \Sigma \cdot a^T 
  $$
   $ \Sigma$ = Covariance matrix of risk factor.
   
   $w$ = vector of risk factors sensitivities
   
   $a$ = vector of sensitivities $ a^{(k)} $

In [44]:
cov_stocks = allriskfactors[relevant_factors[:4]].cov()
cov_swap = allriskfactors[matching_tenors].cov()

var_stocks = np.dot(np.dot(stock_sensitivity, cov_stocks), stock_sensitivity.T)
var_swap = np.dot(np.dot(sofr_sensitivity[matching_tenors].mean() / 10000, cov_swap), (sofr_sensitivity[matching_tenors].mean() / 10000).T)  # 縮放 PV01
portfolio_variance = var_stocks + var_swap

print("Portfolio Variance:", portfolio_variance)

Portfolio Variance: 401349068295.4591


## 4. VAR model: 

## Parametric VaR Model

$$
\text{VaR} = z_{\alpha} \times \sqrt{\mathbf{w}^{T} \Sigma \mathbf{w} }
$$

Parametric VAR Model Formula:
  $$
  VaR_{\alpha,1} = |\mu_P + z_{1-\alpha} \cdot \sigma_P|
  $$


note: for the parametric VAR model's $ \mu_P $ and $ \sigma_P $, can directly use the above 3-4. Portfolio Mean & 3-5. Portfolio Variance calculation result formula to compute Parametric VAR Model's Expected P&L and Standard Deviation:
- Expected P&L ($ \mu_P $):
  $$
  \mu_P = \sum_{k=1}^{m} a^{(k)} \cdot \mu_k \cdot 1
  $$
  $ \mu_k $ = Mean of daily changes in each risk factor.
   $ h = 1 $ day 

- Portfolio Standard Deviation ($ \sigma_P $):
$$
  \sigma_P^2 = V\left[ \sum_{k=1}^{m} a^{(k)} \cdot \Delta x_{0,1d}^{(k)} \right] =  w \cdot \Sigma h \cdot w^T 
=  a \cdot \Sigma \cdot a^T 
  $$
   $ \Sigma$ = Covariance matrix of risk factor.
   
   $w$ = vector of risk factors sensitivities
   
   $a$ = vector of sensitivities $ a^{(k)} $


## Monte Carlo VaR Model - Full revaluation Approach

>##### Fit historical data to normal distribution

In [46]:
allriskfactors

,AAPL daily return,MSFT daily return,F daily return,BAC daily return,1D,1M,2M,3M,6M,9M,...,15Y,16Y,17Y,18Y,19Y,20Y,25Y,30Y,35Y,40Y
2022-11-01,-0.017698,-0.017207,0.002241,0.004430,0.000413,0.000302,0.000216,0.000188,0.000272,0.000444,...,-0.000349,-0.000375,-0.000395,-0.000411,-0.000423,-0.000434,-0.000446,-0.000372,-0.000314,-0.000230
2022-11-02,-0.038019,-0.036009,-0.025700,-0.003043,0.000344,0.000262,0.000214,0.000128,0.000035,0.000132,...,0.000053,0.000019,-0.000013,-0.000040,-0.000058,-0.000064,-0.000018,-0.000043,0.000054,0.000130
2022-11-03,-0.043330,-0.026938,0.015198,-0.005557,0.000440,0.000299,0.000250,0.000302,0.000397,0.000527,...,0.000366,0.000368,0.000388,0.000409,0.000420,0.000409,0.000184,0.000198,0.000115,0.000047
2022-11-04,-0.001949,0.032782,0.018678,0.024767,0.005576,0.002758,-0.000555,-0.000543,-0.000069,-0.000355,...,0.000466,0.000488,0.000494,0.000495,0.000498,0.000512,0.000613,0.000424,0.000315,0.000278
2022-11-07,0.003895,0.028850,0.013966,0.005962,-0.005741,-0.003085,0.000665,0.000813,0.000243,0.000437,...,0.000431,0.000463,0.000487,0.000504,0.000512,0.000514,0.000498,0.000577,0.000577,0.000562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-24,0.002540,0.003668,-0.007871,-0.003919,0.000015,0.000007,0.000011,0.000019,0.000052,0.000083,...,-0.000161,-0.000174,-0.000183,-0.000189,-0.000196,-0.000203,-0.000255,-0.000298,-0.000283,-0.000242
2023-10-25,-0.013583,0.030217,0.013083,0.003136,-0.000124,-0.000027,0.000057,0.000066,0.000059,0.000100,...,0.001196,0.001212,0.001226,0.001237,0.001244,0.001246,0.001202,0.001119,0.001126,0.001147
2023-10-26,-0.024913,-0.038236,-0.016601,0.022064,0.000066,0.000032,-0.000041,-0.000076,-0.000182,-0.000304,...,-0.001020,-0.001024,-0.001027,-0.001028,-0.001025,-0.001015,-0.000895,-0.000769,-0.000804,-0.000877
2023-10-27,0.007938,0.005839,-0.130641,-0.037049,-0.000059,-0.000049,-0.000033,-0.000046,-0.000082,-0.000099,...,0.000191,0.000217,0.000240,0.000259,0.000275,0.000287,0.000327,0.000396,0.000407,0.000384


In [31]:
mu_all = allriskfactors.mean()
sigma_all = allriskfactors.std()  
corr_mat = allriskfactors.corr().values  

>##### Ensure normality by transforming historical data

In [32]:
from statistics import NormalDist

In [33]:
n_mc = 10000
n_factors = len(allriskfactors.columns)  # Number of risk factors (stocks + SOFR)

# Step 3: Generate independent standard normal variables
uniforms = np.random.uniform(size=(n_mc, n_factors))  # Uniform variables
snorms = np.array([[NormalDist().inv_cdf(u) for u in row] for row in uniforms])
# Convert to standard normal

# Step 4: Apply Cholesky decomposition to induce correlation
factor_loadings = np.linalg.cholesky(corr_mat)  # Cholesky decomposition
snorms_correlated = np.dot(snorms, factor_loadings.T)  # Correlated risk factor changes

# Step 5: Transform standard normal samples into actual risk factor returns
simulated_returns = mu_all.values + sigma_all.values * snorms_correlated

# Step 6: Convert to DataFrame with correct column names
simulated_risk_factors = pd.DataFrame(simulated_returns, columns=allriskfactors.columns)

simulated_risk_factors.head()

,AAPL daily return,MSFT daily return,F daily return,BAC daily return,1D,1M,2M,3M,6M,9M,...,15Y,16Y,17Y,18Y,19Y,20Y,25Y,30Y,35Y,40Y
0,-0.031228,-0.035112,-0.007035,-0.002325,-0.000535,-0.000579,-0.000321,-0.000150,-0.000623,-0.000965,...,-0.000287,-0.000246,-0.000207,-0.000173,-0.000144,-0.000121,-0.000076,-0.000045,-0.000083,-0.000050
1,-0.039122,-0.011083,0.015062,-0.003437,-0.001551,-0.000686,0.000278,0.000110,-0.000653,-0.000839,...,-0.000253,-0.000261,-0.000265,-0.000267,-0.000270,-0.000275,-0.000307,-0.000242,-0.000227,-0.000243
2,-0.010742,-0.032628,-0.001276,0.012221,0.000561,0.000134,-0.000592,-0.000314,0.000057,0.000024,...,-0.000720,-0.000743,-0.000763,-0.000781,-0.000797,-0.000811,-0.000845,-0.000835,-0.000799,-0.000751
3,0.006375,0.008400,0.024360,0.010955,-0.000679,0.000033,0.000440,0.000367,0.000372,0.000563,...,0.000688,0.000673,0.000660,0.000647,0.000628,0.000601,0.000423,0.000344,0.000182,0.000091
4,0.004573,0.000667,-0.003153,0.003621,0.000687,0.000606,0.000158,-0.000124,-0.000359,-0.000391,...,-0.000505,-0.000500,-0.000493,-0.000482,-0.000470,-0.000454,-0.000395,-0.000409,-0.000212,-0.000055


>##### Compute 1-Day Portfolio PnL for Each Simulation

$$
\text{PnL}_{\text{total}} = \text{PnL}_{\text{stocks}} + \text{PnL}_{\text{swap}}
$$


$$
\text{PnL}_{\text{swap}} = \text{PV}_{t} - \text{PV}_{t-1}
$$


#### Standard Monte Carlo Simulation 

In [34]:
# stock pnl
def pnl1d_full(simulated_risk):
    # Define the columns that correspond to stock daily returns
    stock_cols = ["AAPL daily return", 
                  "MSFT daily return", 
                  "F daily return", 
                  "BAC daily return"]
    stock_pnl = (simulated_risk[stock_cols].values * 1_000_000).sum(axis=1)
    # Calculate stock PnL: Scale daily returns by a $1M notional per stock and sum across all stocks

    #swap pnl
    swap_pnl = np.array([
        calculate_swap_pv(
            base_rates + simulated_risk[matching_tenors].iloc[i].values,# Apply interest rate shocks
            tenors, # Swap tenors from 1Y to 10Y
            swap_notional, # Notional amount of the swap
            strike # Swap fixed rate
        ) - base_pv  # PnL = PV_new - PV_old(change in present value)
        for i in range(n_mc) # Loop through each Monte Carlo simulation
    ])

    total_pnl = stock_pnl + swap_pnl
    # Total PnL is the sum of stock and swap PnL
    return stock_pnl, swap_pnl, total_pnl

# Set up initial swap valuation parameters
# Extract base rates from SOFR yield curve on a specific date (October 30, 2023)
base_rates = sofr.loc["2023-10-30"][matching_tenors].values.astype(float) 

# Define the swap tenors (1Y, 2Y, ..., 10Y)
tenors = np.arange(1, 11)         # 1Y, 2Y, ..., 10Y

# Define swap notional: $100 million
swap_notional = 100_000_000       # $100 million

# Define swap notional: $100 million
strike = 0.042                    # 4.2%

# Compute the initial swap present value (PV) before applying shocks
base_pv = calculate_swap_pv(base_rates, tenors, swap_notional, strike)

stock_pnl, swap_pnl, simulated_pnl = pnl1d_full(simulated_risk_factors)
# =================================================================

# Compute 1-day 95% Value at Risk (VaR) using Monte Carlo simulations
var1d_full_mc_95 = np.abs(np.percentile(simulated_pnl, 5))  # 5th percentile of the PnL distribution
print(f"VaR [1d, 95%], Full Revaluation: ${var1d_full_mc_95:,.2f}")

VaR [1d, 95%], Full Revaluation: $951,270.45


## Monte Carlo VaR Model - Sensitivity-Based Approach

>##### Step 1 : Define Sensitivities ($𝑎^ (𝑘)$)

In [47]:
a_k=ak_sensitivity
a_k

array([3.99716547e+06, 7.69066531e+06, 1.11655970e+07, 1.44255919e+07,
       1.74700654e+07, 2.03058689e+07, 2.29387143e+07, 2.53711812e+07,
       2.76083034e+07, 7.35762223e+08, 1.00000000e+06, 1.00000000e+06,
       1.00000000e+06, 1.00000000e+06])

>##### Step 2 : Compute Risk Factor Covariance Matrix (Σ)

Then, the resulting covariance matrix will have a shape of **(14 × 14)**:

\begin{bmatrix}
\text{cov\_stocks} & 0 \\
0 & \text{cov\_swap}
\end{bmatrix}


In [ ]:
cov_total = np.block([
    [cov_stocks, np.zeros((4, len(matching_tenors)))], 
    #top-left: stock covariance, top-right:zero matrix
    [np.zeros((len(matching_tenors), 4)), cov_swap] 
    #bottom-left:zero matrix, bottom_right:SOFR covariance
])  

>##### Step 3: Monte Carlo Simulation of Risk Factor Changes (Δ𝑋)

In [37]:
# Step 1: Generate Monte Carlo Simulations for Risk Factor Changes
n_mc = 100000  # Number of simulations
factor_loadings = np.linalg.cholesky(cov_total)  # Cholesky decomposition
simulated_risk_changes = np.random.randn(n_mc, len(a_k)) @ factor_loadings.T  # Correlated risk factor changes

# Step 2: Compute Sensitivity-Based PnL for Each Simulation
def pnl1d_sen(simulated_risk):
    return np.dot(a_k, simulated_risk)  # L = sum(a_k * ΔX)

# Generate PnL samples
pnl1d_sen_sample = [pnl1d_sen(s) for s in simulated_risk_changes]

# Step 3: Compute 1-day 95% Monte Carlo VaR (Extracting 5th percentile)
var1d_sen_mc = np.abs(np.percentile(pnl1d_sen_sample, 5))

# Print Results
print(f"Monte Carlo Sensitivity-Based Portfolio 95% VaR: ${var1d_sen_mc:,.2f}")


Monte Carlo Sensitivity-Based Portfolio 95% VaR: $1,388,501.49
